In [ ]:
%matplotlib inline

import matplotlib
import matplotlib.pyplot as plt
import numpy as np

import blib
blib.useTheme("dark")

import src.radarkit.map as map

In [ ]:
r_earth = 6378.0
origin = (-97.44, 35.22)
view_range = 120
view_range_deg = np.degrees(view_range / r_earth)
lat_min, lat_max = origin[1] - view_range_deg, origin[1] + view_range_deg
lon_min, lon_max = origin[0] - view_range_deg, origin[0] + view_range_deg

In [ ]:
poly = map.get('county')

In [ ]:
coords = []
w = poly['transform']['scale']
b = poly['transform']['translate']
for arc in poly['arcs']:
    lat = w[1] * arc[0][1] + b[1]
    if lat < -89:
        continue
    p = []
    point = [0, 0]
    for p in arc:
        point[0] += p[0]
        point[1] += p[1]
        lon = w[0] * point[0] + b[0]
        lat = w[1] * point[1] + b[1]
        p.append([lon, lat])
    coords.append(np.array(p))

In [ ]:
# View projection matrix

rot = map.makeRotationForCoord(*origin)

def project_v2(coords):
    a, b = coords[:, 0], coords[:, 1]
    m = r_earth * np.cos(a)
    y = r_earth * np.sin(a)
    z = m * np.cos(b)
    x = m * np.sin(b)
    p = np.array((x, y, z)).transpose()
    return np.matmul(p, rot)

def project(coords):
    m = r_earth * np.cos(coords[:, 1])
    y = r_earth * np.sin(coords[:, 1])
    z = m * np.cos(coords[:, 0])
    x = m * np.sin(coords[:, 0])
    p = np.array((x, y, z)).transpose()
    return np.matmul(p, rot)

In [ ]:
def subset_coords(coords, extent):
    x_min, x_max, y_min, y_max = extent
    subset = []
    for p in coords:
        outside = np.logical_or(np.logical_or(p[:, 0] < x_min, p[:, 0] > x_max),
                                np.logical_or(p[:, 1] < y_min, p[:, 1] > y_max))
        if np.all(outside):
            continue
        subset.append(p)
    return subset


def subset_coords_reduce(coords, extent):
    x_min, x_max, y_min, y_max = extent
    subset = []
    for p in coords:
        outside = np.logical_or.reduce((p[:, 0] < x_min, p[:, 0] > x_max, p[:, 1] < y_min, p[:, 1] > y_max))
        if np.all(outside):
            continue
        subset.append(p)
    return subset

subset = subset_coords(coords, (lon_min, lon_max, lat_min, lat_max))
# subset = subset_coords_reduce(coords, (lon_min, lon_max, lat_min, lat_max))

print(f"{len(coords)} -> {len(subset)}")

In [ ]:
%timeit county_lines = subset_coords(coords, (lon_min, lon_max, lat_min, lat_max))
%timeit county_lines = subset_coords_reduce(coords, (lon_min, lon_max, lat_min, lat_max))

In [ ]:
def subset_paths(coords, extent):
    x_min, x_max, y_min, y_max = extent
    subset = []
    for k, c in enumerate(coords):
        p = project(np.radians(c))
        outside = np.logical_or(np.logical_or(p[:, 0] < x_min, p[:, 0] > x_max),
                                np.logical_or(p[:, 1] < y_min, p[:, 1] > y_max))
        if np.all(outside):
            continue
        subset.append(p)
    return subset

subset2 = subset_paths(coords, (-160, 160, -90, 90))

print(f"{len(coords)} -> {len(subset2)}")

In [ ]:
fig = plt.figure()
ax = fig.add_axes([0, 0, 1, 1])
for p in subset:
    ax.add_line(matplotlib.lines.Line2D(p[:, 0], p[:, 1]))
ax.axis('equal')
# ax.set(xlim=(x_min, x_max), ylim=(y_min, y_max))

In [ ]:
n = 0
for p in subset2:
    n += p.shape[0] + 1

subset3 = np.empty((n, 3), dtype=np.float32)

k = 0
for p in subset2:
    n = p.shape[0]
    subset3[k:k+n, :] = p
    k += n + 1

subset3.shape

In [ ]:
fig = plt.figure()
ax = fig.add_axes([0, 0, 1, 1])
for p in subset2:
    ax.add_line(matplotlib.lines.Line2D(p[:, 0], p[:, 1]))
ax.axis('equal')
ax.set(xlim=(lon_min, lon_max), ylim=(lat_min, lat_max))


In [ ]:
plt.close()

In [ ]:
k = 7
fig = plt.figure()
ax = fig.add_axes([0, 0, 1, 1])
for p in subset2:
    ax.add_line(matplotlib.lines.Line2D(p[:, 0], p[:, 1]))
ax.add_line(matplotlib.lines.Line2D(subset2[k][:, 0], subset2[k][:, 1], color='r'))
ax.axis('equal')
ax.set(xlim=(-160, 160), ylim=(-90, 90))

In [ ]:
plt.close()